In [61]:
from pathlib import Path
import os
def load_env_vars_from_file(path: Path):
    file = path.read_text()
    for line in file.splitlines():
        if line.startswith("export"):
            var_to_value = line.split(" ")[1].split("=")
            var_ = var_to_value[0]
            value_ = var_to_value[1][1:-1]
            os.environ[var_] = value_
def set_up_prod_credentials():
    file = "../../../../scripts/prod_setup_creds.sh"
    load_env_vars_from_file(Path(file))
set_up_prod_credentials()

In [62]:
from exchange.interface import ExchangeInterface

e = ExchangeInterface(is_test_run=False)

In [112]:
from datetime import datetime
from helpers.types.portfolio import GetFillsRequest
import pytz
start = datetime(2024, 9, 1).astimezone(pytz.UTC)
fills = e.get_fills(GetFillsRequest(min_ts=int(start.timestamp())))

In [113]:
fills[0]

OrderFill(action=<TradeType.SELL: 'sell'>, count=30, created_time=datetime.datetime(2024, 9, 18, 12, 44, 12, 964591, tzinfo=TzInfo(UTC)), is_taker=True, no_price=57, order_id='d404e238-248e-4fe8-a5b3-13c009518c39', side=<Side.NO: 'no'>, ticker='FED-24SEP-T5.00', trade_id='aa06cef0-ecda-4115-b8de-770daa3839d6', yes_price=43)

In [114]:
from collections import defaultdict
from dataclasses import dataclass

from helpers.types.orders import Side, TradeType

@dataclass
class ProfitData():
    positions: int = 0 
    spent: int = 0 
    made: int = 0 
# From market ticker to position
pd = defaultdict(lambda: ProfitData())
# Had maker fills
maker_tickers = set()

for fill in fills:
    if fill.ticker in maker_tickers:
        continue
    if fill.is_taker == True:
        if fill.ticker in pd:
            del pd[fill.ticker]
        maker_tickers.add(fill.ticker)
        continue
    # For some reason, the fill sides are backwards
    side = fill.side if fill.action == TradeType.BUY else fill.side.get_other_side()
    if (side == Side.YES and fill.action == TradeType.BUY) or (side == Side.NO and fill.action == TradeType.SELL):
        pd[fill.ticker].spent += (fill.yes_price) * (fill.count)
        if pd[fill.ticker].positions < 0:
            pd[fill.ticker].made += fill.count * 100
        pd[fill.ticker].positions += fill.count
    else:
        pd[fill.ticker].spent += (fill.no_price) * (fill.count)
        if pd[fill.ticker].positions > 0:
            pd[fill.ticker].made += fill.count * 100
        pd[fill.ticker].positions -= fill.count



In [115]:
pd

defaultdict(<function __main__.<lambda>()>,
            {'HIGHNY-24SEP16-B75.5': ProfitData(positions=10, spent=1474, made=1000),
             'HIGHNY-24SEP16-B73.5': ProfitData(positions=-10, spent=890, made=0),
             'HIGHCHI-24SEP15-B89.5': ProfitData(positions=-10, spent=4710, made=4000),
             'HIGHNY-24SEP16-B79.5': ProfitData(positions=10, spent=1150, made=1000),
             'HIGHAUS-24SEP16-B99.5': ProfitData(positions=-10, spent=1700, made=1000),
             'HIGHAUS-24SEP16-B97.5': ProfitData(positions=0, spent=2040, made=2000),
             'OSCARNOMACTO-25-TC': ProfitData(positions=-10, spent=440, made=0),
             'RTSPEAKNOEVIL-85': ProfitData(positions=0, spent=3570, made=5000),
             'MCSILENTHILL2-84': ProfitData(positions=-10, spent=590, made=0),
             'HIGHCHI-24SEP16-B86.5': ProfitData(positions=-2, spent=112, made=0),
             'BTCMAXY-24DEC31-80000': ProfitData(positions=-10, spent=700, made=0),
             'DEBATES24-2': Pro

In [116]:

from helpers.types.markets import MarketResult


final_profit = dict()
for ticker, data in pd.items():
    profit = data.made - data.spent
    if data.positions != 0:
        try:
            m = e.get_market(ticker)
            fi = e.get_fills(GetFillsRequest(ticker=ticker))
        except Exception as ex:
            print(ex)
            continue
        fi.sort(key=lambda x: x.created_time)
        # Dont consider this market if there were fills before the start time 
        if len(fi) > 0 and fi[0].created_time < start:
            continue
        if m.result == MarketResult.NO and data.positions < 0:
            profit += abs(data.positions) * 100
        elif m.result == MarketResult.YES and data.positions > 0:
            profit += abs(data.positions) * 100
        else:
            # This market has not settled yet
            continue 
    final_profit[ticker] = profit


404 Client Error: Not Found for url: https://trading-api.kalshi.com/trade-api/v2/markets/GTEMP-24-1.02/23


In [117]:
final_profit

{'HIGHNY-24SEP16-B73.5': 110,
 'HIGHAUS-24SEP16-B99.5': 300,
 'HIGHAUS-24SEP16-B97.5': -40,
 'RTSPEAKNOEVIL-85': 1430,
 'HIGHCHI-24SEP16-B86.5': 88,
 'HIGHCHI-24SEP16-B84.5': 270,
 'HIGHCHI-24SEP16-T84': 340,
 'HIGHMIA-24SEP16-B90.5': 770,
 'TORNADO-24SEP-50': 810,
 'TORNADO-24SEP-25': 400,
 'HIGHCHI-24SEP16-B88.5': 408,
 'HIGHAUS-24SEP15-B96.5': 320,
 'HIGHCHI-24SEP15-B87.5': 370,
 'HIGHMIA-24SEP15-B90.5': 780,
 'TSAW-24SEP15-A2.35': 440,
 'HIGHCHI-24SEP14-B89.5': 80}

In [118]:
sorted(final_profit.items(), key=lambda x: x[1])

[('HIGHAUS-24SEP16-B97.5', -40),
 ('HIGHCHI-24SEP14-B89.5', 80),
 ('HIGHCHI-24SEP16-B86.5', 88),
 ('HIGHNY-24SEP16-B73.5', 110),
 ('HIGHCHI-24SEP16-B84.5', 270),
 ('HIGHAUS-24SEP16-B99.5', 300),
 ('HIGHAUS-24SEP15-B96.5', 320),
 ('HIGHCHI-24SEP16-T84', 340),
 ('HIGHCHI-24SEP15-B87.5', 370),
 ('TORNADO-24SEP-25', 400),
 ('HIGHCHI-24SEP16-B88.5', 408),
 ('TSAW-24SEP15-A2.35', 440),
 ('HIGHMIA-24SEP16-B90.5', 770),
 ('HIGHMIA-24SEP15-B90.5', 780),
 ('TORNADO-24SEP-50', 810),
 ('RTSPEAKNOEVIL-85', 1430)]

[('RECSSNBER-25', -32362),
 ('NEWOUTBREAK-P-24', -6975),
 ('WTAX-25-DEC31', -4700),
 ('NYTOAISETTLE-24DEC31', -4182),
 ('FED-24SEP-T5.00', -3210),
 ('FEDDECISION-24SEP-C25', -3080),
 ('HIGHCHI-24SEP16-B90.5', -1760),
 ('HIGHNY-24SEP14-B82.5', -1450),
 ('BTCATH-24-DEC31-73265.12', -1140),
 ('RTJOKER2-55', -1090),
 ('HIGHMIA-24SEP16-T93', -870),
 ('538APPROVE-24SEP18-B39.5', -770),
 ('RATECUTCOUNT-24DEC31-T3', -760),
 ('RATECUTCOUNT-24DEC31-T5', -720),
 ('HIGHCHI-24SEP15-B89.5', -710),
 ('RTJOKER2-60', -710),
 ('XCEOCHANGE-24DEC31', -710),
 ('BTCMAXY-24DEC31-80000', -700),
 ('TROPSTORM-24DEC01-T15', -700),
 ('DEBATES24-3', -698),
 ('FEDRATEMIN-24DEC31-T4.50', -670),
 ('DEBATES24-2', -653),
 ('RAINNYCM-24SEP-2', -650),
 ('MCSILENTHILL2-84', -590),
 ('MORTGAGEDEF-24-Q3-1.75', -560),
 ('FEDDECISION-24SEP-C26', -554),
 ('RTTRANSFORMERSONE-90', -520),
 ('HIGHNY-24SEP16-B75.5', -474),
 ('RAINNYCM-24SEP-2.5', -460),
 ('OSCARNOMACTO-25-TC', -440),
 ('HIGHAUS-24SEP14-B95.5', -410),
 ('SWITCH2-24', -400),
 ('HIGHAUS-24SEP14-B99.5', -382),
 ('PAYROLLS-24SEP-T142000', -380),
 ('CMEATBAN-25-TN', -368),
 ('RTTRANSFORMERSONE-85', -360),
 ('HIGHAUS-24SEP15-B100.5', -358),
 ('USTESTSMATH-24-SD', -355),
 ('RTTERRIFIER3-75', -340),
 ('HIGHCHI-24SEP14-B87.5', -340),
 ('OSCARCOUNTDUNE2-3', -330),
 ('538APPROVE-24SEP18-B41.0', -320),
 ('538APPROVEMAX-24SEP30-T43', -300),
 ('RTAGATHAALLALONG-85', -300),
 ('538APPROVE-24SEP18-B40.0', -243),
 ('SPACEXCOUNT-24-130', -240),
 ('TSAW-24SEP15-A2.30', -190),
 ('HIGHAUS-24SEP15-B98.5', -186),
 ('HIGHMIA-24SEP15-B92.5', -180),
 ('HIGHNY-24SEP16-B79.5', -150),
 ('HIGHNY-24SEP15-B79.5', -140),
 ('HIGHCHI-24SEP15-B85.5', -140),
 ('OSCARCOUNTDUNE2-4', -140),
 ('538APPROVE-24SEP18-B41.5', -120),
 ('OSCARNOMPIC-25-B', -117),
 ('OSCARCOUNTDUNE2-7', -110),
 ('USTESTSMATH-24-NSD', -100),
 ('USTESTSMATH-24-SI', -70),
 ('HIGHNY-24SEP15-B81.5', -45),
 ('HIGHAUS-24SEP16-B97.5', -40),
 ('HIGHMIA-24SEP15-T93', -33),
 ('HIGHAUS-24SEP16-B95.5', -20),
 ('HIGHCHI-24SEP14-B89.5', 80),
 ('HIGHCHI-24SEP16-B86.5', 88),
 ('HIGHNY-24SEP16-B73.5', 110),
 ('HIGHCHI-24SEP16-B84.5', 270),
 ('HIGHAUS-24SEP16-B99.5', 300),
 ('HIGHAUS-24SEP15-B96.5', 320),
 ('HIGHCHI-24SEP16-T84', 340),
 ('HIGHCHI-24SEP15-B87.5', 370),
 ('HIGHNY-24SEP15-B77.5', 381),
 ('TORNADO-24SEP-25', 400),
 ('HIGHCHI-24SEP16-B88.5', 408),
 ('TSAW-24SEP15-A2.35', 440),
 ('HIGHMIA-24SEP16-B90.5', 770),
 ('HIGHNY-24SEP14-B80.5', 776),
 ('HIGHMIA-24SEP15-B90.5', 780),
 ('TORNADO-24SEP-50', 810),
 ('RTSPEAKNOEVIL-85', 1430),
 ('HIGHNY-24SEP16-B77.5', 1510),
 ('HIGHAUS-24SEP14-B97.5', 1586),
 ('HIGHNY-24SEP14-B84.5', 1942)]

In [119]:
f = list(e.get_fills(GetFillsRequest(ticker="HIGHAUS-24SEP16-B97.5")))

In [120]:
f = sorted(f, key = lambda x: x.created_time)

In [121]:
for fill in f:
    print(fill.action, fill.side, fill.count, fill.yes_price, fill.no_price, fill)

TradeType.BUY Side.NO 10 46¢ 54¢ action=<TradeType.BUY: 'buy'> count=10 created_time=datetime.datetime(2024, 9, 15, 19, 7, 52, 204196, tzinfo=TzInfo(UTC)) is_taker=False no_price=54 order_id='51e2eba8-5258-452f-b9c3-2039e2d829f9' side=<Side.NO: 'no'> ticker='HIGHAUS-24SEP16-B97.5' trade_id='daa433eb-a67c-45a2-91ea-36a51c308272' yes_price=46
TradeType.BUY Side.NO 10 48¢ 52¢ action=<TradeType.BUY: 'buy'> count=10 created_time=datetime.datetime(2024, 9, 15, 20, 40, 39, 327849, tzinfo=TzInfo(UTC)) is_taker=False no_price=52 order_id='2484c971-6024-4d3e-8816-de5281d9fcfb' side=<Side.NO: 'no'> ticker='HIGHAUS-24SEP16-B97.5' trade_id='69de4e1e-6330-44df-80ad-a7ec6ea8beab' yes_price=48
TradeType.BUY Side.YES 20 49¢ 51¢ action=<TradeType.BUY: 'buy'> count=20 created_time=datetime.datetime(2024, 9, 15, 21, 36, 32, 668990, tzinfo=TzInfo(UTC)) is_taker=False no_price=51 order_id='9d11570a-6b1a-4cbc-8ad9-511e300b32f6' side=<Side.YES: 'yes'> ticker='HIGHAUS-24SEP16-B97.5' trade_id='d8b9c399-2da1-477

In [36]:
pd['NEWOUTBREAK-P-24']

ProfitData(positions=-75, spent=6975, made=0)

In [38]:
fills

[OrderFill(action=<TradeType.BUY: 'buy'>, count=82, created_time=datetime.datetime(2024, 6, 11, 21, 14, 0, 570088, tzinfo=TzInfo(UTC)), is_taker=False, no_price=89, order_id='5b8b2f5f-39bd-43e3-aabc-abd5db10f905', side=<Side.YES: 'yes'>, ticker='NEWOUTBREAK-P-24', trade_id='b5aea985-44eb-48d1-83c2-c23e5deb7ffc', yes_price=11),
 OrderFill(action=<TradeType.SELL: 'sell'>, count=7, created_time=datetime.datetime(2024, 6, 12, 21, 22, 45, 315828, tzinfo=TzInfo(UTC)), is_taker=False, no_price=88, order_id='4438650c-fc54-4e9a-96d9-0958d9c3998a', side=<Side.NO: 'no'>, ticker='NEWOUTBREAK-P-24', trade_id='9861fb08-de7b-4fcc-a099-718946bae841', yes_price=12),
 OrderFill(action=<TradeType.SELL: 'sell'>, count=15, created_time=datetime.datetime(2024, 6, 17, 22, 42, 22, 636111, tzinfo=TzInfo(UTC)), is_taker=False, no_price=88, order_id='4438650c-fc54-4e9a-96d9-0958d9c3998a', side=<Side.NO: 'no'>, ticker='NEWOUTBREAK-P-24', trade_id='42de1b32-cf0a-4221-8413-e3e837bb1a32', yes_price=12),
 OrderFill(a